In [1]:
import netCDF4
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
import datetime
import os
import xarray as xr

In [2]:
nc_file = os.listdir("./data/")[0]

In [3]:
nc = netCDF4.Dataset("./data/"+nc_file, mode='r')


In [4]:
nc.variables.keys()

dict_keys(['XDim', 'YDim', 'ColumnAmountNO2Trop'])

In [5]:
nc.variables

{'XDim': <class 'netCDF4._netCDF4.Variable'>
 float64 XDim(XDim)
     standard_name: longitude
     long_name: longitude
     units: degrees_east
     axis: X
 unlimited dimensions: 
 current shape = (30,)
 filling off, 'YDim': <class 'netCDF4._netCDF4.Variable'>
 float64 YDim(YDim)
     standard_name: latitude
     long_name: latitude
     units: degrees_north
     axis: Y
 unlimited dimensions: 
 current shape = (28,)
 filling off, 'ColumnAmountNO2Trop': <class 'netCDF4._netCDF4.Variable'>
 float32 ColumnAmountNO2Trop(YDim, XDim)
     _FillValue: -1.2676506e+30
     missing_value: -1.2676506e+30
     Description: Field=ColumnAmountNO2Trop,StdField=ColumnAmountNO2TropStd,SolarZenithAngle=[0:85],VcdQualityFlags=~19,XTrackQualityFlags=~[1:254],TerrainReflectivity=[0:1000]
     MissingValue: -1.2676506e+30
     Offset: 0.0
     ScaleFactor: 1.0
     Title: NO2 tropospheric column density
     Units: molec/cm2
     fullnamepath: /HDFEOS/GRIDS/ColumnAmountNO2/Data Fields/ColumnAmountNO2Tro

In [6]:
lon = nc.variables["XDim"][:]
lat = nc.variables["YDim"][:]
NO2 = nc.variables["ColumnAmountNO2Trop"][:]

In [7]:
date_parse = nc_file[31-7:33-7]  + "/" + nc_file[33-7:35-7] +  "/" + nc_file[26-7:30-7]
format_str = '%m/%d/%Y' # The format
datetime_obj = datetime.datetime.strptime(date_parse, format_str)
date = datetime_obj.date()

In [8]:
lon.shape

(30,)

In [9]:
lat.shape

(28,)

In [10]:
NO2.shape

(28, 30)

In [11]:
nc = xr.open_dataset("./data/"+nc_file)
df = nc.ColumnAmountNO2Trop.to_dataframe()
#.to_csv(nc_file+'.csv')


In [12]:
df.head()

ColumnAmountNO2Trop
YDim   XDim                        
36.125 -77.625         2.871103e+15
       -77.375         2.815080e+15
       -77.125         2.109011e+15
       -76.875         1.934238e+15
       -76.625         1.993001e+15

In [13]:
df = df.reset_index()

In [14]:
df['Date'] = date

In [15]:
df.index.name = 'id'

In [16]:
df = df.rename(columns={'YDim': 'lat', 'XDim': 'lon'})

In [17]:
df.head()

,lat,lon,ColumnAmountNO2Trop,Date
id,,,,
0,36.125,-77.625,2.871103e+15,2016-11-10
1,36.125,-77.375,2.815080e+15,2016-11-10
2,36.125,-77.125,2.109011e+15,2016-11-10
3,36.125,-76.875,1.934238e+15,2016-11-10
4,36.125,-76.625,1.993001e+15,2016-11-10


In [18]:
"""features = df.apply(
    lambda row: Feature(geometry=Point((float(row['YDim']), float(row['XDim'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = df.drop(['YDim', 'XDim'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)
"""

"features = df.apply(\n    lambda row: Feature(geometry=Point((float(row['YDim']), float(row['XDim'])))),\n    axis=1).tolist()\n\n# all the other columns used as properties\nproperties = df.drop(['YDim', 'XDim'], axis=1).to_dict('records')\n\n# whole geojson object\nfeature_collection = FeatureCollection(features=features, properties=properties)\n"

In [19]:
"""with open("Finished/" + nc_file + ".json", 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)
    """

'with open("Finished/" + nc_file + ".json", \'w\', encoding=\'utf-8\') as f:\n    json.dump(feature_collection, f, ensure_ascii=False)\n    '

In [20]:
comb = pd.DataFrame()
format_str = '%m/%d/%Y' # The format
count =0
for inFile in os.listdir("./data/"):
    date_parse = inFile[31-7:33-7]  + "/" + inFile[33-7:35-7] +  "/" + inFile[26-7:30-7]
    datetime_obj = datetime.datetime.strptime(date_parse, format_str)
    date = datetime_obj.date()
    nc = xr.open_dataset("./data/"+inFile)
    df = nc.ColumnAmountNO2Trop.to_dataframe()
    df = df.reset_index()
    df['Date'] = date
    df = df.rename(columns={'YDim': 'lat', 'XDim': 'lon'})
    comb = comb.append(df)
    count = count + 1
comb.index.name = 'id'
comb = comb.reset_index()
comb = comb.drop("id", axis = 1)
comb = comb.reset_index()
comb.index.name = 'id'


In [23]:
comb = comb.interpolate("cubic")

In [25]:
comb.to_csv("Finished/NO2_Combined.csv")

In [ ]:
comb.head()